In [ ]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.path as pth
import xarray as xr
import sys
import os
os.chdir('/home/561/rmh561/croco/') 

import quadrilateral_tools as intp

In [ ]:
# for testing, restrict to a much smaller ocean grid:
# native shape of PAC12 ocean is 505*1601
olonL = 1601; olatL = 505
alonL = 522; alatL = 168
xfo1 = 0.15; xfoL = 0.25
yfo1 = 0.3; yfoL = 0.34

xfa1 = 0.15; xfaL = 0.27
yfa1 = 0.3; yfaL = 0.36

xo1 = int(olonL*xfo1); xoL = int(olonL*xfoL);
yo1 = int(olatL*yfo1); yoL = int(olatL*yfoL);

xa1 = int(alonL*xfa1); xaL = int(alonL*xfaL);
ya1 = int(alatL*yfa1); yaL = int(alatL*yfaL);

print('ocean x: %03d to %03d, ocean y: %03d to %03d' % (xo1,xoL,yo1,yoL))
print('atm x: %03d to %03d, atm y: %03d to %03d' % (xa1,xaL,ya1,yaL))

In [ ]:
##### Input files #########

iodir = '/g/data/e14/rmh561/croco/data/OASIS_FILES/PAC12_75_cpl/' # Input repository

## OCE ##
fileo = iodir+'croco_grd.nc' # Ocean grid file

## ATM ##
filea = iodir+'wrf_grd.nc' # Atmosphere grid file
filemask= iodir +'wrf_grd.nc' # where to find CPLMASK
extdom = 0  # which external domain should we select in CPLMASK

# Mask smoothed area:
mask_smth = True
filesmthmask= iodir + 'smooth_mask_tanh.nc'
if mask_smth:
    maskname = '_tanhmask'
else:
    maskname = ''

### Do you want to perform test? (plot without smoothing/save netcdf sst smoothed)
test_flag=1
sstfile=iodir+'sst.nc' # sst file
field ='temp_surf' # var to perform test
nrec,zlevel= 0,-1 #

# pixels at a distance of more than 3sig are small enough to be
# considered effectively zero
onedimX = True # If true, only filter in x.
sigx = 24
sigy = 3
cutratio = 0.1

## Output file ##
foldout=iodir # where OASIS_FILES should be
if onedimX:
    fileout = 'mapping_o2a_Xgauss' + maskname + '_sigx'+'{:02d}'.format(int(sigx))+'_cut'+'{:02d}'.format(int(cutratio*100))+'.nc'
    sstfileout = 'sst_interp_smoothXgauss' + maskname +'_sigx'+'{:02d}'.format(int(sigx))+'_cut'+'{:02d}'.format(int(cutratio*100))+'.nc'
else:
    fileout = 'mapping_o2a_gauss' + maskname + '_sigx'+'{:02d}'.format(int(sigx))+'_sigy'+'{:02d}'.format(int(sigy))+'_cut'+'{:02d}'.format(int(cutratio*100))+'.nc'
    sstfileout = 'sst_interp_smoothgauss' + maskname + '_sigx'+'{:02d}'.format(int(sigx))+'_sigy'+'{:02d}'.format(int(sigy))+'_cut'+'{:02d}'.format(int(cutratio*100))+'.nc' 

In [ ]:
# check if OASIS_FILES exits
CHECK_FOLDER = os.path.isdir(foldout)
# If doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(foldout)
    print("created folder : ", foldout)
else:
    print(foldout, "folder already exists.")
##########

def lonlat_to_m(lon,lat,plon,plat):
    # follow Haversine formula
    lon  = lon*2.*np.pi/360.
    lat  = lat*2.*np.pi/360.
    plon = plon*2*np.pi/360.
    plat = plat*2*np.pi/360
    if isinstance(lon,float):
        dx = np.arccos(np.sin(lat)*np.sin(plat) + np.cos(lat)*np.cos(plat)*np.cos(lon-plon))*6371000.
    else:
        dx = np.arccos(np.sin(lat[:])*np.sin(plat) + np.cos(lat[:])*np.cos(plat)*np.cos(lon[:]-plon))*6371000.
    return dx

In [ ]:
# croco grid  

ncf=nc.Dataset(fileo,'r')
lono = ncf.variables['lon_rho'][1:-1,1:-1]
lato = ncf.variables['lat_rho'][1:-1,1:-1]
msko = ncf.variables['mask_rho'][1:-1,1:-1]
ncf.close()

# Restrict data for testing:
lono = lono[yo1:yoL,xo1:xoL]
lato = lato[yo1:yoL,xo1:xoL]
msko = msko[yo1:yoL,xo1:xoL]

[jpio,jpjo]=np.shape(lono)
jpijo = jpio * jpjo

# wrf grid

ncf=nc.Dataset(filea,'r')
lona = np.squeeze(ncf.variables['XLONG'][:])
lata = np.squeeze(ncf.variables['XLAT'][:])
ncf.close

ncf=nc.Dataset(filemask,'r')
mska = np.squeeze(ncf.variables['CPLMASK'][0,extdom,:,:])
ncf.close()

if mask_smth:
    ncf=nc.Dataset(filesmthmask,'r')
    smthmask = np.squeeze(ncf.variables['SMOOTHMASK'][:,:])
    ncf.close()

# Restrict data for testing:
lona = lona[ya1:yaL,xa1:xaL]
lata = lata[ya1:yaL,xa1:xaL]
mska = mska[ya1:yaL,xa1:xaL]
if mask_smth:
    smthmask = smthmask[ya1:yaL,xa1:xaL]

[jpia,jpja]=np.shape(lona)

# Fix longitude cyclicity:
for ii in range(jpja):
    if (lona[0,ii]<=0):
        lona[:,ii] = lona[:,ii] + 360.

# extend theta domain...
# longitude
dx = lona[-1,:] - lona[-2,:]
lona = np.row_stack((lona,lona[-1,:]+dx))
lona = np.c_[lona,lona[:,-1]]

# latitude
lata = np.row_stack((lata, lata[-1,:]))
dy = lata[:,-1]-lata[:,-2]
lata = np.c_[lata,lata[:,-1]+dy]

[jpia,jpja]=np.shape(lona)
jpija = jpia * jpja

# mask
mska = np.c_[mska,np.zeros(jpia-1)]
mska = np.row_stack(( mska,np.zeros(jpja) ))

if mask_smth:
    smska = np.c_[smthmask,np.zeros(jpia-1)]
    smska = np.row_stack((smska,np.zeros(jpja) ))
#---------------------------------------------------------
# compute weights and addresses for bilinear interpolation from oce to atm
#---------------------------------------------------------

weio2a = np.zeros([jpija, 4])
addo2a = np.zeros([jpija, 4])

#
# define all cells that have corners located at lono, lato
# we define the cell with the address of all corners
#
#            3  bdy2 2
#             +------+
#             |      |
#         bdy3|      |bdy1
#             |      |
#             +------+
#            0  bdy0 1
#
celladdr0 = np.arange(0,jpijo).reshape(jpio,jpjo)
# remove last line/colomn  
celladdr0 = celladdr0[0:-1,0:-1].ravel()

xx0 = lono.ravel()[celladdr0]
yy0 = lato.ravel()[celladdr0]
celladdr1 = celladdr0 + 1
xx1 = lono.ravel()[celladdr1]
yy1 = lato.ravel()[celladdr1]
celladdr2 = celladdr1 + jpjo
xx2 = lono.ravel()[celladdr2]
yy2 = lato.ravel()[celladdr2]
celladdr3 = celladdr2 - 1
xx3 = lono.ravel()[celladdr3]
yy3 = lato.ravel()[celladdr3]

In [ ]:
# Understanding the reordering:
python_addr = np.arange(0,6).reshape(2,3)
print(python_addr)
fortran_addr = np.arange(0,6).reshape(3,2).T
print(fortran_addr)

python_add = [1, 2, 3, 4, 5, 0]
fortran_add = fortran_addr.ravel()[python_add]
print(python_add)
print(fortran_add)

In [ ]:
# Plot grids:
ncf=nc.Dataset(sstfile,'r')
sst=np.squeeze(ncf.variables[field][0,1:-1,1:-1])
ncf.close()
sst = sst[yo1:yoL,xo1:xoL]
    
fig = plt.figure(figsize=(15,15))

plt.subplot(2,1,1)
if mask_smth:
    plt.pcolor(lona,lata,smthmask)
else:
    plt.pcolor(lona,lata,lona)
plt.pcolor(lono,lato,sst)
plt.colorbar()

In [ ]:
print(' ocean shape: ' + str(np.shape(lono)))
print(' atm shape: ' + str(np.shape(lona)))

In [ ]:
print('Begin creating weight')

perc=0
for ii in range(0,jpija):
    if (100*ii/jpija >= perc ): print("%02d%% done..." % perc );sys.stdout.flush();perc+=1
    xx = lona.ravel()[ii]
    yy = lata.ravel()[ii]
# Find for closest points
#        dist=lonlat_to_m(xx0,yy0,xx,yy).ravel()
    dist=np.sqrt((xx-xx0)**2+(yy-yy0)**2)      
    for jj in range(0,8): # loop on 4 closest points
        distok=np.argmin(dist).astype('int')
        dist[distok]=1e9 # put high number instead of NAN as argmin is faster ( about 3 times) than nanargmin

        xx0ok = xx0[distok]
        yy0ok = yy0[distok]
        xx1ok = xx1[distok]
        yy1ok = yy1[distok]
        xx2ok = xx2[distok]  
        yy2ok = yy2[distok]
        xx3ok = xx3[distok]
        yy3ok = yy3[distok]

# Check if point is in quad
        polygon=np.array((np.hstack((xx0ok,xx1ok,xx2ok,xx3ok,xx0ok)),np.hstack((yy0ok,yy1ok,yy2ok,yy3ok,yy0ok)) )).T
        points = np.array(( xx.ravel(),yy.ravel() )).T

        isin = pth.Path(polygon,closed=True).contains_points(points) 
        if isin : 
            ind=distok
#   
            xtmp0=np.array([xx0ok]); ytmp0=np.array([yy0ok])
            xtmp1=np.array([xx1ok]); ytmp1=np.array([yy1ok])
            xtmp2=np.array([xx2ok]); ytmp2=np.array([yy2ok])
            xtmp3=np.array([xx3ok]); ytmp3=np.array([yy3ok])
            xtmp =np.array([xx])   ; ytmp =np.array([yy])

            xy=intp.quadrilateral2square(xtmp0,ytmp0,xtmp1,ytmp1,xtmp2,ytmp2,xtmp3,ytmp3,xtmp,ytmp)
            xy = np.round(1000.*xy)/1000.

# define 4 interpolating corners address (with sorted address: 0,1,3,2)
            addtmp = celladdr0[ind] 
            addo2a[ii, 0] = addtmp
            addo2a[ii, 1] = addtmp+1
            addo2a[ii, 2] = addtmp+jpjo
            addo2a[ii, 3] = addtmp+1+jpjo

# define 4 interpolating corners weight
            weio2a[ii, 0] = (1. - xy[0]) * (1. - xy[1])
            weio2a[ii, 1] =       xy[0]  * (1. - xy[1])
            weio2a[ii, 2] = (1. - xy[0]) *       xy[1]
            weio2a[ii, 3] =       xy[0]  *       xy[1]
            break
print('End building weight inside oce grid')

In [ ]:
# Plot total weights before outside ocean grid:
totwei = np.sum(weio2a, 1) #sum over column
bad = np.argwhere(totwei==0); nbad=len(bad) # check where 0
plt.pcolor(np.reshape(totwei,[jpia,jpja]))
plt.colorbar()
#np.shape(totwei.ravel())

In [ ]:
print('Building weight outside oce grid')

totwei = np.sum(weio2a, 1) #sum over column
bad = np.argwhere(totwei.ravel()==0); nbad=len(bad) # check where 0

del dist, distok
if nbad != 0:
    perc=0
    for ii in range(0,nbad):
        if (100*ii/nbad > perc ): print("%02d%% done..." % perc );sys.stdout.flush();perc+=1

        xx = lona.ravel()[int(bad[ii])]
        yy = lata.ravel()[int(bad[ii])]

   # Find 4 closest ocean points 
        distok=np.empty(4)
        distfin=np.empty(4)
#            dist=lonlat_to_m(lono.ravel(),lato.ravel(),xx,yy)
        dist=np.sqrt((xx-lono.ravel())**2+(yy-lato.ravel())**2)
        for kk in range(0,distok.shape[0]):
            distok[kk] = np.argmin(dist).astype('int')
            distfin[kk]=dist[distok[kk].astype('int')]
            dist[distok[kk].astype('int')]=1e5

        # Check if any zero-distance points:
        if (np.all(distfin)): # No
            addo2a[bad[ii], :] = np.sort(distok)
            weio2a[bad[ii], :] = 1./np.sqrt(distfin[np.argsort(distok)])
            weio2a[bad[ii], :] = np.round(1000.*weio2a[bad[ii], :])/1000.
        else: # Yes -> Just use that point.
            kk0 = np.argmin(distfin)
            addo2a[bad[ii], 0] = distok[kk0]
            weio2a[bad[ii], 0] = 1.
            weio2a[bad[ii], 1:] = 0.

print('End building weight outside oce grid')

# normalize weio2a
totweir = 1./np.sum(weio2a,1)
weio2a = weio2a * np.outer(totweir,np.ones(4) )

del dist,distfin,distok,bad,xx0,yy0,xx1,yy1,xx2,yy2,xx3,yy3

In [ ]:
# Plot total weights after outside ocean grid:
totwei = np.sum(weio2a, 1) #sum over column
bad = np.argwhere(totwei==0); nbad=len(bad) # check where 0
plt.pcolor(np.reshape(totwei,[jpia,jpja]))
plt.colorbar()
#np.shape(totwei.ravel())

In [ ]:
#---------------------------------------------------------
# 2) smooth atm values...
#---------------------------------------------------------

# compute weights and addresses for the smooth of atm grid
#   --> addasmth and weiasmth  (jpija, nn*nn)

# pixels at a distance of more than 3sig are small enough to be
# considered effectively zero

nn = int(3*2*sigx+1)

# gaussian window
if onedimX:
    xx = np.arange(0,nn)             # (nn) 
    dd = np.sqrt( ((xx-int(nn/2))**2)/sigx**2 )
    ww = 1./(np.sqrt(np.pi)*sigx) *np.exp( -dd**2 )   # (nn, nn)
    keepww = np.argwhere((ww.ravel() >= max(ww.ravel())*cutratio)); nnok=len(keepww)

    if mask_smth:
        # Add no-smoothed weights version:
        ww_nosmth = np.zeros(np.shape(ww))
        ww_nosmth[int(nn/2)] = 1.

    winaddx = np.arange(0,nn)-int(nn/2)   # (nn)
    winaddy = np.zeros(nn)     # (nn)   
else:
    xx = np.outer(np.arange(0,nn), np.ones(nn))             # (nn, nn) 
    yy = np.outer(np.ones(nn), np.arange(0,nn))            # (nn, nn)
    dd = np.sqrt( ((xx-int(nn/2))**2)/sigx**2 + ((yy-int(nn/2))**2)/sigy**2 )
    ww = 1./(2*np.pi*sigx*sigy) *np.exp( -0.5*dd**2 )   # (nn, nn)
    keepww = np.argwhere((ww.ravel() >= max(ww.ravel())*cutratio)); nnok=len(keepww)

    if mask_smth:
        # Add no-smoothed weights version:
        ww_nosmth = np.zeros(np.shape(ww))
        ww_nosmth[int(nn/2),int(nn/2)] = 1.

    winaddx = np.outer(np.arange(0,nn)-int(nn/2),np.ones(nn))   # (nn, nn)
    winaddy = np.outer(np.ones(nn),np.arange(nn)-int(nn/2))     # (nn, nn)

ww_save = np.copy(ww)
ww_nosmth_save = np.copy(ww_nosmth)
if mask_smth:
    ww_nosmth = ww_nosmth.ravel()[keepww.astype('int')]
ww = ww.ravel()[keepww.astype('int')]                  # (nnok)
winaddx = winaddx.ravel()[keepww.astype('int')]        # (nnok)
winaddy = winaddy.ravel()[keepww.astype('int')]        # (nnok)

In [ ]:
# Plot weights:
fig = plt.figure(figsize=(15,15))

if onedimX:
    plt.subplot(2,1,1)
    plt.plot(ww_save)

    if mask_smth:
        plt.subplot(2,1,2)
        plt.plot(ww_nosmth_save)

else:
    plt.subplot(2,1,1)
    plt.pcolor(ww_save)
    plt.colorbar()
    if mask_smth:
        plt.subplot(2,1,2)
        plt.pcolor(ww_nosmth_save)
        plt.colorbar()

In [ ]:
# 3) merge bilinear interpolation and smooth
#---------------------------------------------------------

print('Begin smoothing')
awater = np.argwhere(mska.ravel() == 1); nawater=len(awater)
nbo2a = 4

addx = np.outer(np.ones(jpia),np.arange(jpja))    # (jpia, jpja)
addx = np.squeeze(addx.ravel()[awater])           # (nawater)
addy = np.outer(np.arange(jpia),np.ones(jpja))    # (jpia, jpja)
addy = np.squeeze(addy.ravel()[awater])           # (nawater)

if mask_smth:
    smask = np.squeeze(smska.ravel()[awater])

weio2afull = np.zeros([nawater, nbo2a*nnok])
addo2afull = np.zeros([nawater, nbo2a*nnok])

minnei = 10.*nbo2a*nnok
maxnei = -1

perc=0

In [ ]:
ii = 100
if (100*ii/nawater >= perc ): print("%02d%% done..." % perc ); sys.stdout.flush();perc+=1
addxii = winaddx + addx[ii] # (nnok)
addyii = winaddy + addy[ii] # (nnok)

ptok = np.argwhere( (addxii.ravel()>=0)&(addxii.ravel()<jpija-1)&(addyii.ravel()>=0)&(addyii.ravel()<jpia-1)); cntok=len(ptok) # exclude added line of jpia/jpja

neismth = addxii + jpja*addyii       # (nnok) 
neismth = neismth[np.squeeze(ptok)]  # (cntok)
weismth = ww[np.squeeze(ptok)]       # (cntok)

neio = addo2a[np.squeeze(neismth).astype('int'), :].T  # (cntok,nbo2a)
weio = weio2a[np.squeeze(neismth).astype('int'), :].T  # (cntok,nbo2a)

smask_tmp = 0.5
# Apply smoothing mask:
weismth_nosmth = ww_nosmth[np.squeeze(ptok)]
weismth_merge = weismth*smask_tmp + weismth_nosmth*(1.-smask_tmp)

#    weio = weio * ( np.outer(np.ones(nbo2a),weismth_merge ))   # (cntok,nbo2a)

In [ ]:
plt.plot(weismth)
plt.plot(weismth_nosmth)
plt.plot(weismth_merge)
plt.ylim([0,0.2])

In [ ]:
#---------------------------------------------------------
# 3) merge bilinear interpolation and smooth
#---------------------------------------------------------

print('Begin smoothing')
awater = np.argwhere(mska.ravel() == 1); nawater=len(awater)
nbo2a = 4

addx = np.outer(np.ones(jpia),np.arange(jpja))    # (jpia, jpja)
addx = np.squeeze(addx.ravel()[awater])           # (nawater)
addy = np.outer(np.arange(jpia),np.ones(jpja))    # (jpia, jpja)
addy = np.squeeze(addy.ravel()[awater])           # (nawater)

if mask_smth:
    smask = np.squeeze(smska.ravel()[awater])

weio2afull = np.zeros([nawater, nbo2a*nnok])
addo2afull = np.zeros([nawater, nbo2a*nnok])

minnei = 10.*nbo2a*nnok
maxnei = -1

perc=0
for ii in range(0,nawater):

    if (100*ii/nawater >= perc ): print("%02d%% done..." % perc ); sys.stdout.flush();perc+=1
    addxii = winaddx + addx[ii] # (nnok)
    addyii = winaddy + addy[ii] # (nnok)

    ptok = np.argwhere( (addxii.ravel()>=0)&(addxii.ravel()<jpija-1)&(addyii.ravel()>=0)&(addyii.ravel()<jpia-1)); cntok=len(ptok) # exclude added line of jpia/jpja

    neismth = addxii + jpja*addyii       # (nnok) 
    neismth = neismth[np.squeeze(ptok)]  # (cntok)
    weismth = ww[np.squeeze(ptok)]       # (cntok)

    neio = addo2a[np.squeeze(neismth).astype('int'), :].T  # (cntok,nbo2a)
    weio = weio2a[np.squeeze(neismth).astype('int'), :].T  # (cntok,nbo2a)

    if mask_smth:
        # Apply smoothing mask:
        weismth_nosmth = ww_nosmth[np.squeeze(ptok)]
        weismth_merge = weismth*smask[ii] + weismth_nosmth*(1.-smask[ii])

        weio = weio * ( np.outer(np.ones(nbo2a),weismth_merge ))   # (cntok,nbo2a)
    else:
        weio = weio * ( np.outer(np.ones(nbo2a),weismth ))   # (cntok,nbo2a)

    weio = weio * msko.ravel()[neio.astype('int')]       # apply ocean land-mask

    okoce = np.argwhere( weio.ravel() != 0); nnei=len(okoce)

    if nnei == 0:
        ibad = addx[ii]
        jbad = addy[ii]
        print('Problem with atm points',addx[ii],addy[ii])
        continue

    neio = neio.ravel()[np.squeeze(okoce)]
    weio = weio.ravel()[np.squeeze(okoce)]

    if nnei>1 :
        tosort = np.argsort(neio)
        neio = neio[tosort]
        weio = weio[tosort]

    val,uniqind = np.unique(neio,return_index=True)
    neluniq = len(uniqind)

    if neluniq != nnei:
        if neluniq ==1:
            uniqneio = np.array([neio[np.squeeze(uniqind).astype('int')]])
            uniqweio = np.array([weio[np.squeeze(uniqind).astype('int')]])
        else:
            uniqneio = neio[np.squeeze(uniqind).astype('int')]
            uniqweio = weio[np.squeeze(uniqind).astype('int')]
        for nn in range(0,nnei):
            if np.sum(uniqind==nn)==0:
                ind = np.argwhere(uniqneio == neio[nn]) 
                uniqweio[ind.ravel()] = uniqweio[ind.ravel()] + weio[nn]
        nnei = neluniq
        neio = np.copy(uniqneio)
        weio = np.copy(uniqweio)
    else:
        uniqind = -1  # free memory


    r_totwei = 1./np.sum(weio)
    weio = weio * r_totwei

    addo2afull[ii, 0:nnei] = neio
    weio2afull[ii, 0:nnei] = weio

    minnei = min([minnei, nnei])
    maxnei = max([maxnei, nnei])

print('End smoothing')

In [ ]:
#---------------------------------------------------------
# check
#---------------------------------------------------------

if test_flag ==1:
    ncf=nc.Dataset(sstfile,'r')
    sst=np.squeeze(ncf.variables[field][:,1:-1,1:-1])
    ncf.close()

    if np.ndim(sst)==4:
        print('sst var is 4-d taking value at time=', nrec,' , zlevel=',zlevel)
        sst=np.squeeze(sst[nrec,zlevel,:])
    elif np.ndim(sst)==3:
        print('sst var is 3-d. Assuming first index is time. Taking record: ',nrec)
        sst=np.squeeze(sst[nrec,:])
    elif np.ndim(sst)==2:
        print('sst var is 2-d.')
        sst=np.squeeze(sst)

    sstsmth = np.zeros([jpia, jpja])
    sstsmth.ravel()[awater.astype('int')] = np.expand_dims(np.sum( weio2afull * sst.ravel()[addo2afull.astype('int')], 1 ),axis=1)
    ct=cutratio*100
    ncf=nc.Dataset(foldout+'/'+sstfileout,'w')
    ncf.createDimension('eta',jpja-1)
    ncf.createDimension('xi',jpia-1)
    sstn = ncf.createVariable(field,'double',('xi','eta'))
    ncf.variables[field][:]       = sstsmth[:-1,:-1]
    ncf.close()


In [ ]:
fig = plt.figure(figsize=(20,10))

plt.subplot(2,2,1)
plt.pcolor(lono,lato,sst,vmin=26.5,vmax=28.3)
plt.colorbar()

plt.subplot(2,2,2)
plt.pcolor(lona,lata,sstsmth,vmin=26.5,vmax=28.3)
plt.colorbar()

plt.subplot(2,2,4)
plt.pcolor(lona,lata,sstsmth,vmin=26.5,vmax=28.3)
plt.pcolor(lono,lato,sst,vmin=26.5,vmax=28.3)
plt.colorbar()

In [ ]:
#---------------------------------------------------------
# write file
#---------------------------------------------------------

# o -> a

ok = np.argwhere(weio2afull != 0); num_links=len(ok)

addo2afull = addo2afull[ok[:,0],ok[:,1]]
weio2afull = weio2afull[ok[:,0],ok[:,1]]

ncf                 = nc.Dataset(foldout+'/'+fileout , 'w' )

ncf.Python_Program_Name = 'build_weight_smth_crocowrf'
if onedimX:
    ncf.SigmaX_Value    = sigx
    ncf.Filt_Type       = '1D X'
else:
    ncf.SigmaX_Value    = sigx
    ncf.SigmaY_Value    = sigy
    ncf.Filt_Type       = '2D'
if mask_smth:
    ncf.Masking         = 'Yes'
    ncf.Maskfile        = filesmthmask
else:
    ncf.Masking         = 'No'
ncf.Cutration           = cutratio
ncf.Original_Filename   = fileout


ncf.createDimension('num_links', num_links)
ncf.createDimension('num_wgts', 1)
ncf.createDimension('src_grid_size',jpio*jpjo)
ncf.createDimension('dst_grid_size',jpia*jpja)

src_var    = ncf.createVariable('src_address','int',('num_links'))
dst_var    = ncf.createVariable('dst_address','int',('num_links'))
rmp_var    = ncf.createVariable('remap_matrix','double',('num_links','num_wgts'))

ncf.variables['src_address'][:]       = addo2afull + 1
dst = np.outer(awater,np.ones(nbo2a*nnok))
dst = dst[ok[:,0],ok[:,1]]
ncf.variables['dst_address'][:]       = dst + 1
ncf.variables['remap_matrix'][:]       = np.reshape(weio2afull,[num_links,1])
ncf.close()


In [ ]:
DS = xr.open_dataset(foldout+'sst_interp_smoothgauss_nomask_sigx24_sigy01_cut10.nc')
DS2 = xr.open_dataset(foldout+'sst_interp_smoothgauss_sigx24_sigy01c10.nc')

In [ ]:
DS.temp_surf

In [ ]:
(DS.temp_surf-DS2.temp_surf.isel(t=0).rename({'x':'x_rho','y':'y_rho'})).plot()